<a href="https://colab.research.google.com/github/soumik12345/point-cloud-segmentation/blob/inference/notebooks/run_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports


In [1]:
!git clone https://github.com/soumik12345/point-cloud-segmentation -q
!pip install wandb ml_collections -qqq

     |████████████████████████████████| 1.7 MB 7.1 MB/s 
     |████████████████████████████████| 88 kB 8.2 MB/s 
     |████████████████████████████████| 97 kB 9.2 MB/s 
     |████████████████████████████████| 139 kB 86.5 MB/s 
     |████████████████████████████████| 180 kB 86.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [2]:
import sys

sys.path.append("point-cloud-segmentation")

from point_seg import transform_block
from point_seg import ShapeNetCoreLoaderInMemory
from configs import shapenetcore

import json
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px

## Configs

In [3]:
CATEGORY = "Table" #@param ["Airplane", "Bag", "Cap", "Car", "Chair", "Earphone", "Guitar", "Knife", "Lamp", "Laptop", "Motorbike", "Mug", "Pistol", "Rocket", "Skateboard", "Table"] {type:"string"}
MODEL_URL = "https://api.wandb.ai/files/pointnet/pointnet_shapenet_core/3sqgxjkb/model-best.h5" #@param {type:"string"}
CONFIGS = shapenetcore.get_config()
METADATA_URL = "https://github.com/soumik12345/point-cloud-segmentation/releases/download/v0.2/metadata.json"

## Load metadata

In [4]:
metadata_path = tf.keras.utils.get_file(
    origin=METADATA_URL,
    fname=METADATA_URL.split('/')[-1]
)
model_path = tf.keras.utils.get_file(
    origin=MODEL_URL,
    fname=MODEL_URL.split('/')[-1]
)

with open(metadata_path) as json_file:
    metadata = json.load(json_file)

LABELS = metadata[CATEGORY]["lables"]
COLORS = metadata[CATEGORY]["colors"]

88768512/88756808 [==============================] - 0s 0us/step


## Visualization utils

In [5]:
def visualize_data(point_cloud, labels):
    fig = px.scatter_3d(
        pd.DataFrame(
            data={
                "x": point_cloud[:, 0],
                "y": point_cloud[:, 1],
                "z": point_cloud[:, 2],
                "label": labels,
            }
        ),
        x="x",
        y="y",
        z="z",
        color="label",
        labels={"label": "Label"},
        color_discrete_sequence=COLORS,
        category_orders={"label": LABELS},
    )
    fig.show()


def visualize_single_point_cloud(point_clouds, label_clouds, idx):
    label_map = LABELS + ["none"]
    point_cloud = point_clouds[idx]
    label_cloud = label_clouds[idx]
    visualize_data(point_cloud, [label_map[np.argmax(label)] for label in label_cloud])

In [6]:
data_loader = ShapeNetCoreLoaderInMemory(
    object_category=CATEGORY,
    n_sampled_points=CONFIGS.num_points,
)
data_loader.load_data()
_, val_dataset = data_loader.get_datasets(
    val_split=CONFIGS.val_split,
    batch_size=CONFIGS.batch_size,
)

1078935552/1078925405 [==============================] - 12s 0us/step


100%|██████████| 7750/7750 [00:18<00:00, 415.47it/s]


## Load model for inference

In [7]:
segmentation_model = tf.keras.models.load_model(
    model_path,
    custom_objects={
        "OrthogonalRegularizer": transform_block.OrthogonalRegularizer
    },
)
val_data_batch = next(iter(val_dataset))
val_predictions = segmentation_model.predict(val_data_batch[0])

In [8]:
idx = np.random.choice(len(val_data_batch[0]))
print(f"Index selected: {idx}")

# Plotting with ground-truth.
print("***********Ground-truth***********")
visualize_single_point_cloud(val_data_batch[0], val_data_batch[1], idx)

# Plotting with predicted labels.
print("***********Predicted***********")
visualize_single_point_cloud(val_data_batch[0], val_predictions, idx)

Index selected: 5
***********Ground-truth***********


***********Predicted***********
